In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
IMAGE_SIZE = [28, 28]

In [ ]:
#Give dataset path
train_path = '/content/drive/MyDrive/Colab Notebooks/TicTacToe Techne NNN/dataset/Train/TrainTestData/Train_Folder'
test_path = '/content/drive/MyDrive/Colab Notebooks/TicTacToe Techne NNN/dataset/Train/TrainTestData/Test_Folder'

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define function to load images and preprocess them
# VGG 16 accepts images of size (32, 32) so I need to resize the (28, 28) imgs to (32x32)

def load_and_preprocess_images(folder_path, target_size=(32, 32)):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder_path))
    for class_index, class_name in enumerate(class_names):
        class_folder = os.path.join(folder_path, class_name)
        for image_name in os.listdir(class_folder):
            image_path = os.path.join(class_folder, image_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if image is not None:
                image = cv2.resize(image, target_size)  # Resize to (32, 32)
                image = np.stack([image] * 3, axis=-1)  # Convert to RGB by stacking channels
                images.append(image)
                labels.append(class_index)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names

# Load and preprocess train and test images
train_folder = '/content/drive/MyDrive/Colab Notebooks/TicTacToe Techne NNN/dataset/Train/TrainTestData/Train_Folder'
test_folder = '/content/drive/MyDrive/Colab Notebooks/TicTacToe Techne NNN/dataset/Train/TrainTestData/Test_Folder'

X_train, y_train, class_names = load_and_preprocess_images(train_folder)
X_test, y_test, _ = load_and_preprocess_images(test_folder)

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(class_names))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(class_names))

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom layers on top of VGG16
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(len(class_names), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

# Freeze the base model layers (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/TicTacToe Techne NNN/dataset/XO_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Data augmentation for the training data
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0,
    height_shift_range=0,
    horizontal_flip=True,
    validation_split=0.1  # Split training data for validation
)

# Train the model
train_generator = datagen.flow(X_train, y_train, batch_size=32, subset='training')
validation_generator = datagen.flow(X_train, y_train, batch_size=32, subset='validation')

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[checkpoint, early_stopping],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test accuracy: {test_accuracy:.4f}")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.7706 - loss: 0.4577
Epoch 1: val_accuracy improved from -inf to 0.91667, saving model to /content/drive/MyDrive/Colab Notebooks/TicTacToe Techne NNN/dataset/XO_model.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 29s 442ms/step - accuracy: 0.7719 - loss: 0.4556 - val_accuracy: 0.9167 - val_loss: 0.2125
Epoch 2/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step - accuracy: 0.9117 - loss: 0.1853
Epoch 2: val_accuracy improved from 0.91667 to 0.97059, saving model to /content/drive/MyDrive/Colab Notebooks/TicTacToe Techne NNN/dataset/XO_model.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 27s 469ms/step - accuracy: 0.9119 - loss: 0.1850 - val_accuracy: 0.9706 - val_loss: 0.1369
Epoch 3/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - accuracy: 0.9319 - loss: 0.1619
Epoch 3: val_accuracy did not improve from 0.97059
58/58 ━━━━━━━━━━━━━━━━━━━━ 23s 401ms/step - accuracy: 0.9321 - loss: 0.1616 - val_accuracy: 0.9412 - val_loss: 0.1647
Epoch 4/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 